In [ ]:
import cv2 as cv

In [ ]:
image = cv.imread('people.jpg')
res = cv.resize(image, dsize=(480,720), interpolation=cv.INTER_CUBIC)

In [ ]:
face_class = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_class = cv.CascadeClassifier('haarcascade_eye.xml')

In [ ]:
image_gray = cv.cvtColor(res, cv.COLOR_BGR2GRAY)

In [ ]:
faces = face_class.detectMultiScale(image_gray, 1.3, 5)

for (x,y,w,h) in faces:
    cv.rectangle(res, (x,y), (x+w, y+h), (255, 0, 0), 2)
    roi_gray = image_gray[y:y+h, x:x+w]
    roi_color = res[y:y+h, x:x+w]
    eyes = eye_class.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)

cv.imshow('People', res)

cv.waitKey(0)

cv.destroyWindow('People')


In [ ]:
video = cv.VideoCapture(0)

while True:
    conectado, frame = video.read()
    frame_cinza = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_class.detectMultiScale(frame_cinza, 1.3, 5)

    for (x,y,w,h) in faces:
        cv.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = frame_cinza[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_class.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)

    cv.imshow('Video', frame)

    if cv.waitKey(1) == ord('s'):
        break

video.release()
cv.destroyAllWindows()

In [ ]:
import mediapipe as mp
import cv2 as cv

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
cap = cv.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        succes, image = cap.read()
        if not succes:
            print('Ignorando o frame vazio.')
            continue
        image.flags.writeable = False
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        results = hands.process(image)

        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hands_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hands_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        cv.imshow('Media Pipe Hands', cv.flip(image, 1))
        if cv.waitKey(5) & 0xFF == 27:
            break
cap.release()

In [1]:
import mediapipe as mp
import cv2 

In [2]:
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [3]:
cap = cv2.VideoCapture(0) #Checks for camera
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils
 
#To access speaker through the library pycaw 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0
 
volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    lmList = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb
 
        length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])
        
        
        print(vol,int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 98),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord(' '): #By using spacebar delay will stop
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

-60.273488116312656 54
-51.48763525931714 97
-52.29459429284781 93
-50.9276835844814 100
-50.833311784980125 100
-50.72023225559997 101
-50.52780913710832 102
-50.56775048357584 102
-50.245413147293725 103
-49.84469204364213 105
-49.72236858102643 106
-49.59500276915359 106
-49.01828112109618 109
-49.00712154664296 109
-48.86716992188187 110
-48.707890439276056 111
-47.94546789056716 114
-47.608827848704365 116
-46.87720647648292 120
-46.74767150908431 120
-46.329857984600494 122
-46.14537730579197 123
-45.98679709876808 124
-45.47029098193093 127
-44.81705901613451 130
-44.321346592829514 132
-43.76819221003292 135
-42.69460434925942 140
-42.122452960411245 143
-41.25733810822277 147
-39.824268043769166 154
-38.86668039570803 159
-37.57776228518381 165
-36.23253710402672 172
-35.222827512428864 177
-33.65720863361085 184
-33.40063222488967 186
-32.58745651330553 190
-31.39948187813848 196
-30.66707094101408 199
-30.390147725312893 200
-30.046100359997972 202
-29.853357252639768 203
-2